In [6]:
"""
TSP for the EL bars. 
A fair chunk of this code is shamelessly lifted from Randy Olsen
Thank you! 

https://github.com/rhiever/Data-Analysis-and-Machine-Learning-Projects/blob/master/optimal-road-trip/Computing%20the%20optimal%20road%20trip%20across%20the%20U.S..ipynb
"""

import googlemaps
from datetime import datetime

# load API key from file not included in repo
apikey = None
with open("./key.txt", 'r') as keyfile:
    apikey = keyfile.readline().strip()

#set up API client
gmaps = googlemaps.Client(key=apikey)

#walking to the bars
tmode = "walking"

#read in json of bars to get addresses
import json 

jsonData=None 
with open("bars.json","r") as iFile: 
    jsonData = json.load(iFile)

#Thank you Randy Olsen! 
from itertools import combinations

#Save this data in a space seperated file ala 830 Hacker Rannk 
outfile=open("distances.ssv","w") 
outfile.write("bar1, bar2, seconds, meters\n")
for (point1, point2) in combinations(jsonData,2): 
    address1=point1["address"]  
    bar1=point1["bar"]
    
    address2=point2["address"]
    bar2=point2["bar"]
    
    route = gmaps.distance_matrix(origins=[address1],
                                  destinations=[address2],
                                  mode=tmode,
                                  language="English",
                                  units="metric"
                                 )
    
    secondsApart=route["rows"][0]["elements"][0]["duration"]["value"]
    metersApart=route["rows"][0]["elements"][0]["distance"]["value"]
    
    
    #write all of the things!
    
    outfile.write("{} {} {} {}\n".format(bar1, bar2, secondsApart, metersApart))
    print bar1, bar2, secondsApart, metersApart 

outfile.close()


ricks pts 25 33
{u'status': u'OK', u'rows': [{u'elements': [{u'duration': {u'text': u'1 min', u'value': 25}, u'distance': {u'text': u'33 m', u'value': 33}, u'status': u'OK'}]}], u'origin_addresses': [u'224 Abbot Rd, East Lansing, MI 48823, USA'], u'destination_addresses': [u'210 Abbot Rd, East Lansing, MI 48823, USA']}
ricks elAz 242 329
{u'status': u'OK', u'rows': [{u'elements': [{u'duration': {u'text': u'4 mins', u'value': 242}, u'distance': {u'text': u'0.3 km', u'value': 329}, u'status': u'OK'}]}], u'origin_addresses': [u'224 Abbot Rd, East Lansing, MI 48823, USA'], u'destination_addresses': [u'225 Ann St, East Lansing, MI 48823, USA']}
ricks harpers 80 104
{u'status': u'OK', u'rows': [{u'elements': [{u'duration': {u'text': u'1 min', u'value': 80}, u'distance': {u'text': u'0.1 km', u'value': 104}, u'status': u'OK'}]}], u'origin_addresses': [u'224 Abbot Rd, East Lansing, MI 48823, USA'], u'destination_addresses': [u'131 Albert Ave, East Lansing, MI 48823, USA']}
ricks theRiv 191 259


I really wanted to just brute force things since 14 is a small number and there'd only be 87 trillion possible orders to check... That was a bad idea. 

In [10]:
#storing everything
allBars=set()
distances=dict()

#read in the things
with open("distances.ssv","r") as iFile: 
    #ignore header 
    iFile.readline()
    
    
    for line in iFile: 
        #lazy way to get data
        bar1, bar2, secondsApart, _ = line.strip().split()
        
        #makes looking up distances later easier 
        distances[(bar1,bar2)]=int(secondsApart)
        distances[(bar2,bar1)]=int(secondsApart)
        
        allBars.add(bar1)
        allBars.add(bar2)
        
from itertools import permutations

assert False, "Don't even try this. Just don't."

bestRouteLen=None
bestRoute=None
#brute force through all of the routes
for route in permutations(allBars,len(allBars)): 
    
        
    
    #walk through the route and total the length
    totalLength=0
    for i, bar in enumerate(route): 
        nextBar=route[(i+1)%len(route)]
        
        totalLength+=distances[(bar,nextBar)]

    if not bestRouteLen or totalLength < bestRouteLen: 
        bestRouteLen=totalLength
        bestRoute=route
        
    

    
print(bestRoute)
print(bestRouteLen)
        
        

KeyboardInterrupt: 

Now to use Randy's GP code.

In [1]:
import pandas as pd
import numpy as np

waypoint_distances = {}
waypoint_durations = {}
all_waypoints = set()

with open("distances.ssv","r") as iFile: 
    #ignore header 
    iFile.readline()
    
    for line in iFile: 
        #lazy way to get data
        bar1, bar2, secondsApart, metersApart = line.strip().split()
        
        waypoint_distances[frozenset([bar1, bar2])] = metersApart
        waypoint_durations[frozenset([bar1, bar2])] = secondsApart
        all_waypoints.update([bar1, bar2])
    
    
import random

def compute_fitness(solution):
    """
        This function returns the total distance traveled on the current road trip.
        
        The genetic algorithm will favor road trips that have shorter
        total distances traveled.
    """
    
    solution_fitness = 0.0
    
    for index in range(len(solution)):
        waypoint1 = solution[index - 1]
        waypoint2 = solution[index]
        solution_fitness += waypoint_distances[frozenset([waypoint1, waypoint2])]
        
    return solution_fitness

def generate_random_agent():
    """
        Creates a random road trip from the waypoints.
    """
    
    new_random_agent = list(all_waypoints)
    random.shuffle(new_random_agent)
    return tuple(new_random_agent)

def mutate_agent(agent_genome, max_mutations=3):
    """
        Applies 1 - `max_mutations` point mutations to the given road trip.
        
        A point mutation swaps the order of two waypoints in the road trip.
    """
    
    agent_genome = list(agent_genome)
    num_mutations = random.randint(1, max_mutations)
    
    for mutation in range(num_mutations):
        swap_index1 = random.randint(0, len(agent_genome) - 1)
        swap_index2 = swap_index1

        while swap_index1 == swap_index2:
            swap_index2 = random.randint(0, len(agent_genome) - 1)

        agent_genome[swap_index1], agent_genome[swap_index2] = agent_genome[swap_index2], agent_genome[swap_index1]
            
    return tuple(agent_genome)

def shuffle_mutation(agent_genome):
    """
        Applies a single shuffle mutation to the given road trip.
        
        A shuffle mutation takes a random sub-section of the road trip
        and moves it to another location in the road trip.
    """
    
    agent_genome = list(agent_genome)
    
    start_index = random.randint(0, len(agent_genome) - 1)
    length = random.randint(2, 20)
    
    genome_subset = agent_genome[start_index:start_index + length]
    agent_genome = agent_genome[:start_index] + agent_genome[start_index + length:]
    
    insert_index = random.randint(0, len(agent_genome) + len(genome_subset) - 1)
    agent_genome = agent_genome[:insert_index] + genome_subset + agent_genome[insert_index:]
    
    return tuple(agent_genome)

def generate_random_population(pop_size):
    """
        Generates a list with `pop_size` number of random road trips.
    """
    
    random_population = []
    for agent in range(pop_size):
        random_population.append(generate_random_agent())
    return random_population
    
def run_genetic_algorithm(generations=5000, population_size=100):
    """
        The core of the Genetic Algorithm.
        
        `generations` and `population_size` must be a multiple of 10.
    """
    
    population_subset_size = int(population_size / 10.)
    generations_10pct = int(generations / 10.)
    
    # Create a random population of `population_size` number of solutions.
    population = generate_random_population(population_size)

    # For `generations` number of repetitions...
    for generation in range(generations):
        
        # Compute the fitness of the entire current population
        population_fitness = {}

        for agent_genome in population:
            if agent_genome in population_fitness:
                continue

            population_fitness[agent_genome] = compute_fitness(agent_genome)

        # Take the top 10% shortest road trips and produce offspring each from them
        new_population = []
        for rank, agent_genome in enumerate(sorted(population_fitness,
                                                   key=population_fitness.get)[:population_subset_size]):
            
            if (generation % generations_10pct == 0 or generation == generations - 1) and rank == 0:
                print("Generation %d best: %d | Unique genomes: %d" % (generation,
                                                                       population_fitness[agent_genome],
                                                                       len(population_fitness)))
                print(agent_genome)
                print("")

            # Create 1 exact copy of each of the top road trips
            new_population.append(agent_genome)

            # Create 2 offspring with 1-3 point mutations
            for offspring in range(2):
                new_population.append(mutate_agent(agent_genome, 3))
                
            # Create 7 offspring with a single shuffle mutation
            for offspring in range(7):
                new_population.append(shuffle_mutation(agent_genome))

        # Replace the old population with the new population of offspring 
        for i in range(len(population))[::-1]:
            del population[i]

        population = new_population
        
        run_genetic_algorithm(generations=5000, population_size=100)